In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

Requirement already up-to-date: gensim==4.0.0 in /usr/local/lib/python3.7/dist-packages (4.0.0)
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149821 sha256=21a932e9606b0d3702d3bc8759e863070cfd1f221a2c7dec3998210061d98b1b
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [ ]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]


In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-yelp-polarity")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

Framing fasttext attack

In [ ]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [ ]:
def printexample(attack):
  print(attack)
  from collections import deque

  import textattack
  import tqdm
  import time
  from IPython.display import display, HTML
  num_examples = 25
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

In [ ]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification(),MaxWordsPerturbed(max_percent=0.3)]

search_method = GreedyWordSwapWIR(wir_method='weighted-saliency')

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Reusing dataset yelp_polarity (/root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581)
textattack: Loading datasets dataset yelp_polarity, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.3
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)


In [ ]:
printexample(attack)










  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.3
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)











  4%|▍         | 1/25 [00:04<01:42,  4.26s/it]








[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:04<01:42,  4.27s/it]










[Succeeded / Failed / Total] 1 / 0 / 1:   8%|▊         | 2/25 [00:07<01:27,  3.81s/it]








[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:07<01:27,  3.81s/it]










[Succeeded / Failed / Total] 2 / 0 / 2:  12%|█▏        | 3/25 [00:08<01:00,  2.76s/it]








[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:08<01:00,  2.77s/it]

...










[Succeeded / Failed / Total] 18 / 4 / 22:  92%|█████████▏| 23/25 [02:41<00:14,  7.03s/it]








[Succeeded / Failed / Total] 19 / 4 / 23:  92%|█████████▏| 23/25 [02:41<00:14,  7.03s/it]










[Succeeded / Failed / Total] 19 / 4 / 23:  96%|█████████▌| 24/25 [02:50<00:07,  7.12s/it]








[Succeeded / Failed / Total] 20 / 4 / 24:  96%|█████████▌| 24/25 [02:50<00:07,  7.12s/it]










[Succeeded / Failed / Total] 20 / 4 / 24: 100%|██████████| 25/25 [02:52<00:00,  6.91s/it]








[Succeeded / Failed / Total] 21 / 4 / 25: 100%|██████████| 25/25 [02:52<00:00,  6.91s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 16.0%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 12.51% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 238.84 |
+-------------------------------+--------+


textattack: Attack time: 172.80336356163025s


In [ ]:
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attack import Attack
from textattack.transformations import (
    CompositeTransformation,
    WordSwapEmbedding,
    WordSwapHomoglyphSwap,
    WordSwapNeighboringCharacterSwap,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
)

transformation = CompositeTransformation(
            [
                # (1) Insert: Insert a space into the word.
                # Generally, words are segmented by spaces in English. Therefore,
                # we can deceive classifiers by inserting spaces into words.
                WordSwapRandomCharacterInsertion(
                    random_one=True,
                    letters_to_insert=" ",
                    skip_first_char=True,
                    skip_last_char=True,
                ),
                # (2) Delete: Delete a random character of the word except for the first
                # and the last character.
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (3) Swap: Swap random two adjacent letters in the word but do not
                # alter the first or last letter. This is a common occurrence when
                # typing quickly and is easy to implement.
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (4) Substitute-C (Sub-C): Replace characters with visually similar
                # characters (e.g., replacing “o” with “0”, “l” with “1”, “a” with “@”)
                # or adjacent characters in the keyboard (e.g., replacing “m” with “n”).
                WordSwapHomoglyphSwap(),
                # (5) Substitute-W
                # (Sub-W): Replace a word with its topk nearest neighbors in a
                # context-aware word vector space. Specifically, we use the pre-trained
                # GloVe model [30] provided by Stanford for word embedding and set
                # topk = 5 in the experiment.
                WordSwapEmbedding(max_candidates=5),
            ]
        )

constraints = [RepeatModification(), StopwordModification()]
        # In our experiment, we first use the Universal Sentence
        # Encoder [7], a model trained on a number of natural language
        # prediction tasks that require modeling the meaning of word
        # sequences, to encode sentences into high dimensional vectors.
        # Then, we use the cosine similarity to measure the semantic
        # similarity between original texts and adversarial texts.
        # ... "Furthermore, the semantic similarity threshold \eps is set
        # as 0.8 to guarantee a good trade-off between quality and
        # strength of the generated adversarial text."
constraints.append(UniversalSentenceEncoder(threshold=0.8))
        #
        # Goal is untargeted classification
        #
goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
search_method = GreedyWordSwapWIR(wir_method="delete")

attack=Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Reusing dataset yelp_polarity (/root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581)
textattack: Loading datasets dataset yelp_polarity, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)


In [ ]:
printexample(attack)









  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)










  4%|▍         | 1/25 [00:06<02:39,  6.64s/it]







[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:06<02:39,  6.65s/it]









[Succeeded / Failed / Total] 1 / 0 / 1:   8%|▊         | 2/25 [00:09<01:44,  4.56s/it]







[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:09<01:44,  4.56s/it]









[Succeeded / Failed / Total] 2 / 0 / 2:  12%|█▏        | 3/25 [00:09<01:11,  3.27s/it]







[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:09<01:12,  3.27s/it]

...









[Succeeded / Failed / Total] 18 / 4 / 22:  92%|█████████▏| 23/25 [02:48<00:14,  7.34s/it]







[Succeeded / Failed / Total] 19 / 4 / 23:  92%|█████████▏| 23/25 [02:48<00:14,  7.34s/it]









[Succeeded / Failed / Total] 19 / 4 / 23:  96%|█████████▌| 24/25 [02:55<00:07,  7.30s/it]







[Succeeded / Failed / Total] 20 / 4 / 24:  96%|█████████▌| 24/25 [02:55<00:07,  7.30s/it]









[Succeeded / Failed / Total] 20 / 4 / 24: 100%|██████████| 25/25 [02:56<00:00,  7.06s/it]







[Succeeded / Failed / Total] 21 / 4 / 25: 100%|██████████| 25/25 [02:56<00:00,  7.07s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 16.0%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 34.34% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 357.68 |
+-------------------------------+--------+


textattack: Attack time: 176.63516521453857s
